# Nerdle Solver - Initial Guess Assessment Optimization
We prove (by brute-force) that you can always solve mini-Nerdle in at most $4$ guesses regardless of the starting expression, provided you use the optimal strategy. The worst start having repeating numbers and thus less information, e.g. `10-5=5`. The best start has all different numbers: `28/7=4`, which needs at most $3$ guesses and $2.65 \pm 0.5$ guesses.

To find the best initial guess, we map out the game tree.

In [300]:
%load_ext autoreload
%autoreload 2

import collections
import ctypes
import itertools
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt

import analysis
import nerdle
import score as s
import generator
sgo = ctypes.CDLL(s.SCORE_GUESS_OPT_SO)
from nerdle import Hint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initial Guess Optimization

In [345]:
for num_slots in range(6, 9):
    print("num_slots", num_slots)
    db_file = "db/nerdle{}.db".format(num_slots) 
    solver_data = nerdle.create_solver_data(num_slots, db_file)
    d = solver_data.score_db

    print("Building tree")
    tree = analysis.GameTreeBuilder(solver_data).build(debug=True)
    tdc = TreeDepthCalculator(tree)

    # Distribution of #guesses for all answers.
    num_guesses = np.array([depth for node, depth in tdc.depth.items() if not node.children]) + 1
    freq = collections.Counter(num_guesses)
    num_leaves = sum(1 for node in tdc.depth if not node.children)

    print("Best initial guess", solver_data.answers[tree.key], "#distinct games", len(tdc.depth), "#leaves", num_leaves)
    for k, v in sorted(freq.items()):
        print("{} guesses: {:6.2f}%".format(k, 100 * v / num_leaves))
    plt.hist(num_guesses);

num_slots 6
Building tree
 Node[key=(87, '3*8=24', 10), answers=206, score=(206, 206), children=73]
	 Node[key=(145, '6*9=54', 1), answers=9, score=(206, 9), children=9]
	 Node[key=(44, '15-6=9', 1), answers=9, score=(206, 9), children=9]
	 Node[key=(12, '11-2=9', 1), answers=8, score=(206, 8), children=8]
	 Node[key=(3, '10-3=7', 1), answers=8, score=(206, 8), children=8]
	 Node[key=(15, '11-5=6', 1), answers=10, score=(206, 10), children=10]
	 Node[key=(0, '1+9=10', 1), answers=3, score=(206, 3), children=3]
	 Node[key=(10, '10/2=5', 2), answers=10, score=(206, 10), children=9]
	 Node[key=(13, '11-3=8', 1), answers=6, score=(206, 6), children=6]
	 Node[key=(3, '10-3=7', 1), answers=6, score=(206, 6), children=6]
	 Node[key=(0, '1+9=10', 1), answers=2, score=(206, 2), children=2]
	 Node[key=(0, '1+9=10', 1), answers=2, score=(206, 2), children=2]
	 Node[key=None, answers=1, score=(206, 1), children=0]
	 Node[key=None, answers=1, score=(206, 1), children=0]
	 Node[key=None, answers=1, 

AssertionError: 

In [330]:
for k, v in sorted(freq.items()):
    print("{} guesses: {:6.2f}%".format(k, 100 * v / num_leaves))


2 guesses:   1.44%
3 guesses:  57.23%
4 guesses:  41.33%


In [329]:
collections.Counter(num_guesses) / num_leaves

Counter({4: 3125, 3: 4327, 2: 109})

In [328]:
num_guesses = np.array([depth for node, depth in tdc.depth.items() if not node.children]) + 1

In [344]:
for num_slots in range(6, 9):
    print("num_slots", num_slots)
    db_file = "db/nerdle{}.db".format(num_slots) 
    solver_data = nerdle.create_solver_data(num_slots, db_file)
    d = solver_data.score_db
    answers = solver_data.all_keys
    %time bucket_size, _, k = min((max(collections.Counter(d[k]).values()), k not in answers, k) for k in solver_data.all_keys)
    print(solver_data.answers[k], "bucket size", bucket_size, "reduction factor", d.shape[1] / bucket_size)

num_slots 6
CPU times: user 9.67 ms, sys: 67 µs, total: 9.74 ms
Wall time: 9.81 ms
3*8=24 bucket size 10 reduction factor 0.04854368932038835
num_slots 7
CPU times: user 7.34 s, sys: 18.5 ms, total: 7.36 s
Wall time: 7.37 s
24-16=8 bucket size 158 reduction factor 0.020896706784816824
num_slots 8
CPU times: user 42.6 s, sys: 160 ms, total: 42.8 s
Wall time: 42.9 s
58-46=12 bucket size 101 reduction factor 0.005698809456638266
